In [0]:
MERGE INTO dw.fact_delivery
USING (
  SELECT
    d.match_type_number                 AS match_id,
    d.innings_number,
    d."over"                            AS over_number,
    d.ball_in_over,

    -- resolve batting & bowling teams
    tbat.team_id                        AS batting_team_id,
    tbow.team_id                        AS bowling_team_id,

    -- resolve player IDs with team context
    p_bat.player_id                     AS striker_id,
    p_ns.player_id                      AS non_striker_id,
    p_bow.player_id                     AS bowler_id,

    -- measures
    (d.total - d.extras)                AS runs_off_bat,
    d.extras,
    d.total                             AS total_runs,
    CASE WHEN (d.total - d.extras) = 4 THEN TRUE ELSE FALSE END AS is_boundary_4,
    CASE WHEN (d.total - d.extras) = 6 THEN TRUE ELSE FALSE END AS is_boundary_6,
    CASE WHEN d.player_out IS NOT NULL THEN TRUE ELSE FALSE END AS is_wicket,
    d.player_out_kind                   AS wicket_kind,
    d.player_out_fielders               AS fielder_names,

    d.stg_file_hashkey,
    d.ingest_date

  FROM (
    -- Deduplicate if staging ever reloads same delivery multiple times
    SELECT *,
           ROW_NUMBER() OVER (
             PARTITION BY match_type_number, innings_number, "over", ball_in_over
             ORDER BY stg_modified_ts DESC
           ) AS rn
    FROM stg.delivery_table
  ) d
  JOIN dw.dim_match dm
    ON dm.match_id = d.match_type_number
  JOIN dw.dim_team tbat
    ON tbat.team_key = LOWER(TRIM(d.country))      -- batting side
  JOIN dw.dim_team tbow
    ON tbow.team_id = CASE
                        WHEN tbat.team_id = dm.first_team_id THEN dm.second_team_id
                        ELSE dm.first_team_id
                      END
  LEFT JOIN dw.dim_player p_bat
    ON p_bat.player_key = LOWER(TRIM(d.batter)) || '|' || tbat.team_key
  LEFT JOIN dw.dim_player p_ns
    ON p_ns.player_key = LOWER(TRIM(d.non_striker)) || '|' || tbat.team_key
  LEFT JOIN dw.dim_player p_bow
    ON p_bow.player_key = LOWER(TRIM(d.bowler)) || '|' || tbow.team_key
  WHERE d.rn = 1
) s
ON dw.fact_delivery.match_id       = s.match_id
   AND dw.fact_delivery.innings_number = s.innings_number
   AND dw.fact_delivery.over_number    = s.over_number
   AND dw.fact_delivery.ball_in_over   = s.ball_in_over
WHEN MATCHED THEN UPDATE SET
  striker_id       = s.striker_id,
  non_striker_id   = s.non_striker_id,
  bowler_id        = s.bowler_id,
  batting_team_id  = s.batting_team_id,
  bowling_team_id  = s.bowling_team_id,
  runs_off_bat     = s.runs_off_bat,
  extras           = s.extras,
  total_runs       = s.total_runs,
  is_boundary_4    = s.is_boundary_4,
  is_boundary_6    = s.is_boundary_6,
  is_wicket        = s.is_wicket,
  wicket_kind      = s.wicket_kind,
  fielder_names    = s.fielder_names,
  stg_file_hashkey = s.stg_file_hashkey,
  ingest_date      = s.ingest_date
WHEN NOT MATCHED THEN INSERT (
  match_id, innings_number, over_number, ball_in_over,
  striker_id, non_striker_id, bowler_id,
  batting_team_id, bowling_team_id,
  runs_off_bat, extras, total_runs,
  is_boundary_4, is_boundary_6, is_wicket, wicket_kind, fielder_names,
  stg_file_hashkey, ingest_date
)
VALUES (
  s.match_id, s.innings_number, s.over_number, s.ball_in_over,
  s.striker_id, s.non_striker_id, s.bowler_id,
  s.batting_team_id, s.bowling_team_id,
  s.runs_off_bat, s.extras, s.total_runs,
  s.is_boundary_4, s.is_boundary_6, s.is_wicket, s.wicket_kind, s.fielder_names,
  s.stg_file_hashkey, s.ingest_date
);

In [0]:
select * from dw.fact_delivery where match_id = 1927;